**url规律**

`https://maoyan.com/board/4`

`https://maoyan.com/board/4?offset=0`

`https://maoyan.com/board/4?offset=20`

In [1]:
# 获取单页html测试

import requests

# 对requests.get(url).text进行扩展
def get_one_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    return None

# 封装到main()函数
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    print(html)
 
# main()
# 获得html文件

**js运行前的源代码,见 'relatedfiles/maoyan.html'**

In [47]:
# 测试代码 - get html
import json
import requests
from requests.exceptions import RequestException
import time

In [48]:
# 复制代码 全部理解
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

In [ ]:
# 复制代码 非运行代码
def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    print(html)

In [ ]:
# 复制代码 非运行代码
if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)
        time.sleep(1)

In [6]:
# 测试代码 - from html
with open('relatedfiles/maoyan.html', 'r') as t:
    html = t.read()

In [44]:
import re

In [45]:
pattern = r"""
<dd>    # 匹配开头的<dd>
.*?     # 任意内容(懒惰模式-尽可能少重复)

board-index # 定位排名
.*?     #
>       # 所需内容之前: 起始标签的右尖括号>
(.*?)   # 所需内容1: 排名
</i>    # 所需内容之后: 结尾标签</i>
.*?     #

name    # 定位片名
.*?     #
>.*?>   # 所需内容之前: 两个右尖括号>
(.*?)   # 所需内容2: 片名
</a>    # 所需内容之后: 结尾标签</a>
.*?

star    # 定位主演
.*?     #
>.{20}  # 所需内容之前: 右尖括号> (not good)
(.*?)   # 所需内容3: 主演
.{9}</p># 所需内容之后: 结尾标签</p> (not good)
.*?

releasetime # 定位上映时间
.*?     #
>.{5}   # 所需内容之前: 右尖括号> (not good)
(.*?)   # 所需内容4: 上映时间
</p>    # 所需内容之后: 结尾标签</p>
.*?

integer # 定位评分1
.*?     #
>       # 所需内容之前: 右尖括号>
(.*?)   # 所需内容5: 评分1
</i>    # 所需内容之后: 结尾标签</i>
.*?

fraction # 定位评分2
.*?     #
>       # 所需内容之前: 右尖括号>
(.*?)   # 所需内容6: 评分2
</i>    # 所需内容之后: 结尾标签</i>
.*?

</dd>   # 匹配结果的</dd>
"""

In [46]:
items = re.findall(re.compile(pattern, re.S | re.X), html)
items

[('1', '霸王别姬', '张国荣,张丰毅,巩俐', '1993-01-01', '9.', '6'),
 ('2', '肖申克的救赎', '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', '1994-10-14(美国)', '9.', '5'),
 ('3', '罗马假日', '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', '1953-09-02(美国)', '9.', '1'),
 ('4', '这个杀手不太冷', '让·雷诺,加里·奥德曼,娜塔莉·波特曼', '1994-09-14(法国)', '9.', '5'),
 ('5', '教父', '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', '1972-03-24(美国)', '9.', '3'),
 ('6', '泰坦尼克号', '莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', '1998-04-03', '9.', '5'),
 ('7', '唐伯虎点秋香', '周星驰,巩俐,郑佩佩', '1993-07-01(中国香港)', '9.', '2'),
 ('8', '千与千寻', '柊瑠美,入野自由,夏木真理', '2001-07-20(日本)', '9.', '3'),
 ('9', '魂断蓝桥', '费雯·丽,罗伯特·泰勒,露塞尔·沃特森', '1940-05-17(美国)', '9.', '2'),
 ('10', '乱世佳人', '费雯·丽,克拉克·盖博,奥利维娅·德哈维兰', '1939-12-15(美国)', '9.', '1')]

# 完整代码

In [28]:
# 导入库
import json
import requests
from requests.exceptions import RequestException
import time
import re

In [29]:
# 获取html
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

In [30]:
# 解析html
def parse_one_page(html):
    pattern = r"""
    <dd>    # 匹配开头的<dd>
    .*?     # 任意内容(懒惰模式-尽可能少重复)
    
    board-index # 定位排名
    .*?     #
    >       # 所需内容之前: 起始标签的右尖括号>
    (.*?)   # 所需内容1: 排名
    </i>    # 所需内容之后: 结尾标签</i>
    .*?     #
    
    name    # 定位片名
    .*?     #
    >.*?>   # 所需内容之前: 两个右尖括号>
    (.*?)   # 所需内容2: 片名
    </a>    # 所需内容之后: 结尾标签</a>
    .*?
    
    star    # 定位主演
    .*?     #
    >.{20}  # 所需内容之前: 右尖括号> (not good)
    (.*?)   # 所需内容3: 主演
    .{9}</p># 所需内容之后: 结尾标签</p> (not good)
    .*?
    
    releasetime # 定位上映时间
    .*?     #
    >.{5}   # 所需内容之前: 右尖括号> (not good)
    (.*?)   # 所需内容4: 上映时间
    </p>    # 所需内容之后: 结尾标签</p>
    .*?
    
    integer # 定位评分1
    .*?     #
    >       # 所需内容之前: 右尖括号>
    (.*?)   # 所需内容5: 评分1
    </i>    # 所需内容之后: 结尾标签</i>
    .*?
    
    fraction # 定位评分2
    .*?     #
    >       # 所需内容之前: 右尖括号>
    (.*?)   # 所需内容6: 评分2
    </i>    # 所需内容之后: 结尾标签</i>
    .*?
    
    </dd>   # 匹配结果的</dd>
    """
    return re.findall(re.compile(pattern, re.S | re.X), html)

In [31]:
# main
def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    return parse_one_page(html)

In [37]:
result = []
for i in range(10):
    result = result + main(offset=i * 10)
    time.sleep(3)

In [38]:
result

[('1', '霸王别姬', '张国荣,张丰毅,巩俐', '1993-01-01', '9.', '6'),
 ('2', '肖申克的救赎', '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', '1994-10-14(美国)', '9.', '5'),
 ('3', '罗马假日', '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', '1953-09-02(美国)', '9.', '1'),
 ('4', '这个杀手不太冷', '让·雷诺,加里·奥德曼,娜塔莉·波特曼', '1994-09-14(法国)', '9.', '5'),
 ('5', '教父', '马龙·白兰度,阿尔·帕西诺,詹姆斯·肯恩', '1972-03-24(美国)', '9.', '3'),
 ('6', '泰坦尼克号', '莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', '1998-04-03', '9.', '5'),
 ('7', '唐伯虎点秋香', '周星驰,巩俐,郑佩佩', '1993-07-01(中国香港)', '9.', '2'),
 ('8', '千与千寻', '柊瑠美,入野自由,夏木真理', '2001-07-20(日本)', '9.', '3'),
 ('9', '魂断蓝桥', '费雯·丽,罗伯特·泰勒,露塞尔·沃特森', '1940-05-17(美国)', '9.', '2'),
 ('10', '乱世佳人', '费雯·丽,克拉克·盖博,奥利维娅·德哈维兰', '1939-12-15(美国)', '9.', '1'),
 ('11', '天空之城', '寺田农,鹫尾真知子,龟山助清', '1992', '9.', '1'),
 ('12', '喜剧之王', '周星驰,莫文蔚,张柏芝', '1999-02-13(中国香港)', '9.', '2'),
 ('13', '辛德勒的名单', '连姆·尼森,拉尔夫·费因斯,本·金斯利', '1993-12-15(美国)', '9.', '2'),
 ('14', '大闹天宫', '邱岳峰,毕克,富润生', '1965-12-31', '9.', '0'),
 ('15', '音乐之声', '朱莉·安德鲁斯,克里斯托弗·普卢默,埃琳诺·帕克', '1965-03-02(美国)', '9.', '0'),
 ('16', '剪刀手爱德华